## Continu Inzicht Demo (What-if)
> https://tscontinuinzicht.hkvservices.nl/ci-demo

**Imports**

In [1]:
# import algemeen
from datetime import datetime, timedelta, timezone
from pathlib import Path

import pandas as pd

# imports base modules
from toolbox_continu_inzicht import Config, DataAdapter
from toolbox_continu_inzicht.utils.datetime_functions import datetime_from_epoch

d:\Users\Thonus\mijnSoftware\Toolbox Continu Inzicht\toolbox-continu-inzicht\.pixi\envs\default\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

<pre style="font-size: 10pt">
GlobalVariables:
    rootdir: "data"
    calc_time: "2024-11-19"
    moments: [-24,0,24,48]

    LoadsWaterinfo:
            parameters: ["waterhoogte"]
            MISSING_VALUE: 999

    LoadsMatroos:
        website: "noos"
        model: "fews_rmm_km"
        parameters: ["WATHTE"]

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: ""
        parameters: [ "WATHTE [m][NAP][OW]" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsFewsLimburg:
        host: "http://109.232.44.231"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "App_Maas"
        parameters: [ "WATHTE" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: -999

DataAdapter:

    calculation_start_config:
        type: ci_postgresql_to_calculation_start
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    calculation_end_config:
        type: ci_postgresql_to_calculation_end
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    in_measuringstation_waterinfo:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "waterinfo"

    in_measuringstation_matroos:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "matroos"

    in_measuringstation_fews_hhnk:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "fews_hhnk"

    in_measuringstation_fews_limburg:
        type: ci_postgresql_from_measuringstations
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        source: "fews_limburg"

    out_waterstanden_ci_postgresql_001:
        type: ci_postgresql_measuringstation_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 0.01

    out_waterstanden_ci_postgresql_1:
        type: ci_postgresql_measuringstation_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 1

    in_measuringstation_data_table:
        type: ci_postgresql_measuringstation_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    in_measuringstation_conditions_table:
        type: ci_postgresql_from_conditions
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    out_measuringstation_states_table:
        type: ci_postgresql_to_states
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"

    in_section_table:
        type: ci_postgresql_from_sections
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    in_sectionfractions_table:
        type: ci_postgresql_from_sectionfractions
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    out_waterstanden_section_ci_postgresql:
        type: ci_postgresql_section_load_to_data
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"        

    df_moment_waterstanden:
        type: python

    in_fragilitycurves_table:
        type: ci_postgresql_fragilitycurves_table
        database: geoserver
        schema: continuinzicht_demo_realtime
        measureid: 0

    in_measure_fragilitycurves_table:
        type: ci_postgresql_measure_fragilitycurves_table
        database: geoserver
        schema: continuinzicht_demo_realtime                

    in_section_load_from_data_table:
        type: ci_postgresql_section_load_from_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    out_section_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 5

    out_section_technical_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 100

    out_section_measure_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: geoserver
        schema: continuinzicht_demo_realtime
        parameter_id: 101        

    in_section_conditions:
        type: ci_postgresql_section_thresholds_from_conditions_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    in_section_data_failure_probability:
        type: ci_postgresql_section_failure_probability_from_data_table
        database: geoserver
        schema: continuinzicht_demo_realtime

    out_section_states:
        type: ci_postgresql_section_to_states
        database: geoserver
        schema: continuinzicht_demo_realtime
</pre>
</details>

In [2]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-demo-whatif.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

**Aanmaken adapter**

In [3]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Huidig moment (datum/tijd) in what-if kiezen**

In [4]:
# zet de reken datum/tijd van what-if berekening en zet deze globaal in de data_adapter
year = 2024
month = 1
day = 2
hour = 12
minute = 0
second = 0

# LET OP altijd in UTC tijd zetten
calc_time = datetime(year, month, day, hour, minute, second).replace(
    tzinfo=timezone.utc
)

# zet de reken datum/tijd via de data_adapter in de globale variabelen
data_adapter.set_global_variable(key="calc_time", value=calc_time)

print(f"Huidige rekentijd: {calc_time} {calc_time.tzinfo} voor berekening.")

Huidige rekentijd: 2024-01-02 12:00:00+00:00 UTC voor berekening.


**Bepalen datum/tijd per moment**

In [5]:
# lees reken datum/tijd en momenten uit de globale variabelen
calc_time = data_adapter.config.global_variables["calc_time"]
moments = data_adapter.config.global_variables["moments"]

# initialiseer eerste datum/tijd en de laatste datum/tijd
min_date_time = calc_time
max_date_time = calc_time

# definieer een lege tabel met (reken)momenten voor in de CI database (tabel 'moments')
records = []

# stap door alle momenten en bepaal de eerste datum/tijd en de laatste datum/tijd door de reken datum/tijd te corrigeren met de momenten
for moment in moments:
    moment_time = calc_time + timedelta(hours=moment)
    min_date_time = min(min_date_time, moment_time)
    max_date_time = max(max_date_time, moment_time)

    # definieer een (reken)moment voor de CI database (tabel 'moments')
    record = {"moment_id": moment, "date_time": moment_time, "calc_time": moment_time}

    # voeg de rij toe aan de tabel voor de CI database (tabel 'moments')
    records.append(record)

# sla de (reken)momenten op in de CI database (tabel 'moments') via de data_adapter 'out_moments_table'
data_adapter.output(output="out_moments_table", df=pd.DataFrame.from_records(records))

# print feedback naar het scherm: eerste datum/tijd en de laatste datum/tijd
print(f"min: {min_date_time} {min_date_time.tzinfo}")
print(f"max: {max_date_time} {max_date_time.tzinfo}")

min: 2024-01-01 12:00:00+00:00 UTC
max: 2024-01-04 12:00:00+00:00 UTC


**Scenario kiezen**

In [6]:
# definieer het scenario dat gebruikt moet worden
scenario_name = "Voorbeeld1"

# lees reken datum/tijd en moments uit de globale variabelen
calc_time = config.global_variables["calc_time"]
moments = config.global_variables["moments"]

# ophalen van lijst met alle scenario's uit CI database (tabel 'scenarios') via de data_adapter 'ci_postgresql_from_scenarios'
df_scenarios = data_adapter.input(input="ci_from_scenario")

# definieer een lege tabel met de simulatie eigenschappen voor in de CI database (tabel 'simulation')
records = []

# definieer de simulatie eigenschappen voor de CI database (tabel 'simulation')
result = df_scenarios.loc[
    df_scenarios["scenario_name"] == scenario_name,
    [
        "scenario_id",
        "scenario_time_step",
        "scenario_min_date_time",
        "scenario_max_date_time",
    ],
]

if not result.empty:
    scenarioid = result.iloc[0]["scenario_id"]
    timestep = result.iloc[0]["scenario_time_step"]
    min_calc_time = datetime_from_epoch(result.iloc[0]["scenario_min_date_time"])
    max_calc_time = datetime_from_epoch(result.iloc[0]["scenario_max_date_time"])

    # stap door alle momenten en bepaal de eerste datum/tijd en de laatste datum/tijd door de reken datum/tijd te corrigeren met de momenten
    min_date_time = calc_time
    max_date_time = calc_time
    for moment in moments:
        moment_time = calc_time + timedelta(hours=moment)
        min_date_time = min(min_date_time, moment_time)
        max_date_time = max(max_date_time, moment_time)

    starttime = min_date_time
    endtime = max_date_time

    # controleer of momenten binnen bereik van het gekozen what-if scenario vallen
    if starttime > min_calc_time and endtime < max_calc_time:
        record = {
            "id": 1,
            "scenarioid": scenarioid,
            "datetime": calc_time,
            "starttime": starttime,
            "endtime": endtime,
            "timestep": timestep,
            "active": True,
        }

        # voeg de rij toe aan de tabel voor de CI database (tabel 'simulation')
        records.append(record)

        # sla het gekozen scenario op in de CI database (tabel 'simulation')
        data_adapter.output(
            output="out_simulation_table", df=pd.DataFrame.from_records(records)
        )
    else:
        print(
            f"Er gaat iets mis! De berekende momenten op basis van het gekozen huidige moment ({calc_time} {calc_time.tzinfo} ), vallen buiten het bereik van het gekozen scenario!"
        )

else:
    scenarioid = None
    print(
        f"Er gaat iets mis! Het scenario: {scenario_name}, bestaat niet in de CI database!"
    )

**Start berekening**

In [7]:
from toolbox_continu_inzicht.helpers import calculation_start

start, end = calculation_start(
    data_adapter=data_adapter, output="calculation_start_config", calc_time=calc_time
)
print(f"{start} - {end}")

2024-01-01 12:00:00+00:00 - 2024-01-04 12:00:00+00:00


**What-if belastingen inlezen en wegschrijven naar data tabel**

In [20]:
from toolbox_continu_inzicht.loads import LoadsCIWhatIf

loads_whatif = LoadsCIWhatIf(data_adapter=data_adapter)
loads_whatif.run(input="ci_from_loads", output="ci_to_data")

**Van alle belastingen alleen de belastingen van opgegeven momenten halen**

In [21]:
from toolbox_continu_inzicht.loads import LoadsToMoments

load_moments = LoadsToMoments(data_adapter=data_adapter)
load_moments.run(
    input="in_measuringstation_data_table", output="df_moment_waterstanden"
)

if load_moments.df_out is not None:
    df_moments = load_moments.df_out.reset_index(drop=False)
    df_moments["date_time"] = df_moments["date_time"].astype(object)

    # dataframe moments toevoegen aan adapter
    data_adapter.set_dataframe_adapter("df_moment_waterstanden", df_moments)

: 

In [13]:
a = data_adapter.config.data_adapters["df_moment_waterstanden"]
len(a)

9

**Status bepalen voor de meetstations**

In [10]:
from toolbox_continu_inzicht.loads import LoadsClassify

# - via opgegeven waterstandsgrenzen wordt per meetstation de status bepaald

loads_classify = LoadsClassify(data_adapter=data_adapter)
loads_classify.run(
    input=["in_measuringstation_conditions_table", "df_moment_waterstanden"],
    output="out_measuringstation_states_table",
)

**Waterstanden bepalen voor de dijkvakken**

In [ ]:
from toolbox_continu_inzicht.sections import SectionsLoads

# - er wordt via de bovenliggende en onderliggende meetstation
#   en de fractie per meetstation (opgegeven in 'section fraction')
#   bepaald wat de maatgevende waterstand voor een dijkvak is.

if df_moments is not None:
    sections_loads = SectionsLoads(data_adapter=data_adapter)
    sections_loads.run(
        input=[
            "in_section_table",
            "df_moment_waterstanden",
            "in_sectionfractions_table",
        ],
        output="out_waterstanden_section_ci_postgresql",
    )

In [13]:
from toolbox_continu_inzicht.sections import SectionsTechnicalFailureprobability

sections_failureprobability = SectionsTechnicalFailureprobability(
    data_adapter=data_adapter
)
sections_failureprobability.run(
    input=["in_fragilitycurves_table", "in_section_load_from_data_table"],
    output="out_section_technical_failure_probability_data",
)

In [14]:
from toolbox_continu_inzicht.sections import SectionsMeasureFailureprobability

sections_failureprobability = SectionsMeasureFailureprobability(
    data_adapter=data_adapter
)
sections_failureprobability.run(
    input=["in_measure_fragilitycurves_table", "in_section_load_from_data_table"],
    output="out_section_measure_failure_probability_data",
)

In [15]:
from toolbox_continu_inzicht.sections import SectionsFailureprobability

sections_failureprobability = SectionsFailureprobability(data_adapter=data_adapter)
sections_failureprobability.run(
    input="in_section_data_failure_probability",
    output="out_section_failure_probability_data",
)

In [16]:
from toolbox_continu_inzicht.sections import SectionsClassify

sections_classify = SectionsClassify(data_adapter=data_adapter)
sections_classify.run(
    input=["in_section_conditions", "in_section_data_failure_probability"],
    output="out_section_states",
)

In [11]:
from toolbox_continu_inzicht.helpers import calculation_end

calculation_end(data_adapter=data_adapter, output="calculation_end_config")